In [5]:
import os
import openai
from string import Template
import json
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep


### Load Enviroment files

In [6]:
# Load environment variables
load_dotenv()

True

In [8]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"

# openai.api_key = os.getenv("OPENAI_KEY")


# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


In [94]:
def generate_KGOld(initial_entity,iterations):

    #Initialize knowledge graph based on api structure
    knowledge_graph={...} #start with only 1 entity, sustainable energy

    ##initialize a list of entities that already have subgraphs generated
    processed_entities=[]

    for i in iterations: #iterations will control the size of the graph

        #generate list of current entities existing in the knowledge graph
        current_entities=...##generate from the knowlegede grapg

        #sustainble energy - renwwable, gas, eoc

        for entity in current_entities:
            if entity not in processed_entities:

                # generate new subgraph based on this entity
                new_subgraph=generate_subgraph(entity,10) 
                processed_entities.append(entity)

                #  add new entities and new relations to KG
                knowledge_graph=updateKG(knowledge_graph, new_subgraph)

    return knowledge_graph


def generate_subgraphOld(entity,count):
    # Use the openai api to generate a list (of length 'count') of the most related entities
    entities=...##use openAI api

    # Generate the relations between the initial entity and the new list of entities
    relations=...##use openAI api

    # return json of new entities and relations
    new_subgraph=...##entities and relations

    return new_subgraph


def generate_similar_entities(initial_entity, count):
    # Use the OpenAI API to generate similar entities
    prompt=f"Generate {count} entities similar to {initial_entity}. Return as an array only e.g ['entity1','entity2']"
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    similar_entities = response_entities.choices[0].message.content

    # response = openai.Completion.create(
    #     engine="text-davinci-003",
    #     prompt=f"Generate {count} entities similar to {initial_entity}:",
    #     max_tokens=50,
    #     n=count
    # )

    # similar_entities = [choice['text'] for choice in response['choices']]
    return similar_entities


def generate_KG(initial_entity, iterations):
    # Initialize knowledge graph based on the API structure
    knowledge_graph = {initial_entity: {'relations': [], 'subgraph': {}}}

    # Initialize a list of entities that already have subgraphs generated
    processed_entities = []

    for _ in range(iterations):  # iterations will control the size of the graph
        # Generate list of current entities existing in the knowledge graph
        current_entities = generate_similar_entities(initial_entity,10)
        print(f"current_entities === {current_entities}")
            # [entity for entity in knowledge_graph if entity not in processed_entities]
    for entity in current_entities:
        if entity not in processed_entities:
            print(f"Entity processed: {entity}")
            # Generate new subgraph based on this entity
            new_subgraph = generate_subgraph(entity, 10)
            processed_entities.append(entity)
            # print(f"processed_entities=== {processed_entities}")
            
            # Check if the entity exists in the knowledge graph
            if entity not in knowledge_graph:
                knowledge_graph[entity] = {'relations': [], 'subgraph': {}}

                knowledge_graph[entity]['subgraph'] = new_subgraph
                related_entities = new_subgraph.get('entities', [])
                relations = new_subgraph.get('relations', [])
                
                for related_entity in related_entities:
                    if related_entity not in knowledge_graph:
                        knowledge_graph[related_entity] = {'relations': [], 'subgraph': {}}
                    if entity not in knowledge_graph[related_entity]['relations']:
                        knowledge_graph[related_entity]['relations'].append(entity)

                    if related_entity not in knowledge_graph[entity]['relations']:
                        knowledge_graph[entity]['relations'].append(related_entity)

    return knowledge_graph

def generate_subgraph(entity, count):
    # Use the openai api to generate a list (of length 'count') of the most related entities
    prompt = f"What are {count} entities related to {entity}?"
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    entities = response_entities.choices[0].message.content


    prompt_relations = f"What are the relations between {entity} and {', '.join(entities)}?"
    relations = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt_relations},
                    ]
                )

    # relations = relations['answers']
    relations = response_entities.choices[0].message.content


    # return json of new entities and relations
    new_subgraph = {
        "entities": entities,
        "relations": relations
    }

    return new_subgraph


def updateKG(knowledge_graph, new_subgraph):

    ##update knowledge graph by adding new entities and relations from the subgraph

    return knowledge_graph

# Example usage
# initial_entity = "sustainable energy"
# knowledge_graph = generate_KG(initial_entity,4)
# print(knowledge_graph)


In [95]:
# Example usage
initial_entity = "sustainable energy"
knowledge_graph = generate_KG(initial_entity,4)
# print(knowledge_graph)


# Write the knowledge graph to a JSON file
output_file = '../03_Output/01_Auto KGs/00_Current Versions/knowledge_graph.json'
with open(output_file, 'w') as json_file:
    json.dump(knowledge_graph, json_file, indent=4)

print(f"Knowledge graph saved to {output_file}")


current_entities === ['Renewable energy', 'Green technology', 'Clean energy', 'Energy efficiency', 'Circular economy', 'Climate change mitigation', 'Carbon footprint reduction', 'Eco-friendly practices', 'Sustainable transportation', 'Waste management']
current_entities === ['Renewable energy', 'Green technology', 'Clean energy', 'Energy efficiency', 'Circular economy', 'Climate change mitigation', 'Carbon footprint reduction', 'Eco-friendly practices', 'Sustainable transportation', 'Waste management']
current_entities === ['Renewable energy', 'Green technology', 'Clean energy', 'Energy efficiency', 'Circular economy', 'Climate change mitigation', 'Carbon footprint reduction', 'Eco-friendly practices', 'Sustainable transportation', 'Waste management']
current_entities === ['Renewable energy', 'Green technology', 'Clean energy', 'Energy efficiency', 'Circular economy', 'Climate change mitigation', 'Carbon footprint reduction', 'Eco-friendly practices', 'Sustainable transportation', 'Was

KeyboardInterrupt: 